# UCI Introduction to Machine Learning
**Day 3: Graph Neural Networks**


Notebook adapted by Gage DeZoort from a similar notebook offered in his tutorial, [Graph Neural Networks for your Research](https://github.com/GageDeZoort/prc_gnn_tutorial/blob/main/gnn_tutorial.ipynb).



<a href="https://colab.research.google.com/github/GageDeZoort/intro_ml_uci/blob/main/day_2/intro_to_ML_day_3_gnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Software Installation
Just click through the following cells, no need to linger!

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt

# check the torch/cuda installations
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

Given the current PyTorch/Cuda installations, we'll install [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) as follows:

In [ ]:
# install PyTorch Geometric (PyG)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

**PyTorch Geometric (PyG)** is a high-level library for **graph deep learning** built on PyTorch.  
It provides data structures, layers, and utilities to build **GNNs** (Graph Neural Networks) for tasks like node/edge/graph classification, link prediction, and graph regression—at scale.

# 1. Graph-Structured Data

<p align="center">
  <img src="https://raw.githubusercontent.com/GageDeZoort/intro_ml_uci/main/day_3/images/graph.png" width="500">
</p>


A **graph** $G$ is a mathematical object consisting of a set of **nodes** (vertices) $V$ and their pairwise relationships encoded as **edges** $E$, i.e. $G=(V,E)$. In the following we will usually denote $|V|=n_\mathrm{nodes}$ and $|E|=n_\mathrm{edges}$.

Graphs can easily represent a wide range of structured data including **atoms in molecules** (nodes=atoms, edges=molecular bonds), **users in a social network** (nodes=people, edges=social connections), **cities and roads in a transportation system** (nodes=cities, edges=roads), **planetary simulations** (nodes=planets, edges=gravitational interactions), and more.

In PyG, graphs are stored as `Data` objects (see the [docs](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html#torch_geometric.data.Data)). Given a graph named `data`, we will find in many cases that it has several of the following attributes:

- `data.x`: node feature matrix of dimension $n_\mathrm{nodes}\times d_V$, where $d_V$ is the number of features attached to each node.
- `data.edge_index`: a sparse edge list in COO format indicating node indices at each end of each edge; this object has size $2\times n_\mathrm{edges}$.
- `data.edge_attr`: edge feature matrix of dimension $n_\mathrm{edges}\times d_E$, where $d_E$ is the number of features attached to each edges.
- `data.y`: training target with arbitary shape (remember, targets may be node-level, edge-level, or graph-level)

##  1.1 Karate Club
Let's take a look at an example graph in PyG. The Zachary Karate Club dataset (1977) is a social network describing friendships in a university karate club:
- 34 nodes (members of the karate club)
- 78 edges (friendships outside of the club)

A conflict arose between an instructor and an adminstrator, causing a faction in the club - half of the members formed a new karate club around the previous instructor and half either found a new instructor or quit karate.
- Truth labels: faction, 0 or 1 (per node)

Note that this is a single graph, and the goal is to perform *node classification* on the training set of nodes (a subset of all the nodes in the graph). In this way, Karate Club is used for semi-supervised node classification.

In [ ]:
from torch_geometric.datasets import KarateClub

# KarateClub is a small, easily visualized community graph
karate_club = KarateClub()._data

> **Exercise 1.1a**: In the following cell, print the KarateClub object and its attributes, `x`, `edge_index`, `y`.

In [ ]:
# EXERCISE




As a brief aside, let's take a look at how PyG stores edges. In a fully connected graph, there are $\frac{1}{2}n_\mathrm{nodes}(n_\mathrm{nodes}-1)$ edges. Sparse graphs have relatively few edges, i.e. $n_\mathrm{edges}\ll \frac{1}{2}n_\mathrm{nodes}(n_\mathrm{nodes}-1)$. Adjacency matrices aren't necessarily the best way to represent sparse graphs - instead, packages like PyG use edge index lists in COO format.

In [ ]:
# edge lists (COO format)
out_nodes = karate_club.edge_index[0]
print('edges move out of the following nodes:\n', out_nodes)
in_nodes = karate_club.edge_index[1]
print('edges move into the following nodes:\n', in_nodes)

**Exercise 1.1b** Use `torch_geometric.utils.to_dense_adj` to generate an adjacency matrix from `edge_index`.

In [ ]:
# EXERCISE




We can print several additional details about the graph:

In [ ]:
# more details about the graph
def get_graph_stats(data):
  print(f'Number of nodes: {data.num_nodes}')
  print(f'Number of edges: {data.num_edges}')
  print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
  print(f'Number of training nodes: {data.train_mask.sum()}')
  print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
  print(f'Has isolated nodes: {data.has_isolated_nodes()}')
  print(f'Has self-loops: {data.has_self_loops()}')
  print(f'Is undirected: {data.is_undirected()}')

get_graph_stats(karate_club)

Okay, good, we've taken a look at our first instance of `Data`. Now let's visualize the graph; note that here it has 4 truth labels instead of 2, which have been generated by a graph clustering algorithm (if you're interested see [this ref](https://ieeexplore.ieee.org/document/4358966), but it's not important here). We'll use `networkx`, a networks dynamics software package, to visualize the graph.

In [ ]:
import torch_geometric
import networkx as networkx
print('Truth labels', karate_club.y)
karate_club_nx = torch_geometric.utils.to_networkx(karate_club, to_undirected=True)
plt.figure(figsize=(10,10))
nx.draw(karate_club_nx, with_labels=True, node_color=karate_club.y)

## 1.2 ENZYMES Graphs

Karate Club is used for semi-supervised node classification; let's look at a dataset used for graph-level classification. The [ENZYMES dataset](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.TUDataset.html#torch_geometric.datasets.TUDataset)
contains 600 graphs representing proteins with ~32.6 nodes (secondary structure elements - helices, turns or sheets) and ~124.3 edges (denoting neighbors in space or in the amino acid sequence) each. For expert details see the [TUDataset paper](https://arxiv.org/pdf/2007.08663.pdf). The goal is to assign each protein to one of 6 enzyme commission numbers, which describe the chemical reaction the enzyme catalyzes.

In [ ]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='/tmp/TUDataset', name="ENZYMES", use_node_attr=True)
dataset.data

For PyG `datasets` containing multiple graphs, you'll typically want to use a `DataLoader` to load up batches of graphs:

In [ ]:
from torch_geometric.loader import DataLoader

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
loader = DataLoader(dataset, batch_size=32, shuffle=False)
for batch in loader:
  print(batch)

**Exercise 1.1c**: Notice also that the data batches appear to have concatenate the nodes and edges of all the graphs into a single "batch graph." How does this work? (Hint: look at the `batch` attribute appended to each batch.)

In [ ]:
# EXERCISE





# 2. Graph Neural Networks

In general, GNNs work by leveraging **local information** across the graph structure to intelligently compute new features for the nodes (and/or edges). A commonly used class of GNNs is the **Graph Convolutional Network (GCN)**. GCNs use more-or-less the same convolution operations you've seen used in CNNs; however, instead of applying them to patches of images, they apply them to node neighborhoods.

**Message** are computed from each of a node's neighbors, and these messages are passed back to the node and used to update its features. We're going to make this painfully concrete; first, here's a schematic:

![](https://drive.google.com/uc?id=1f5MJO9Kw1tWjJMBJBsZ-M4tMFz_dydr2)

## 2.1 Graph Convolutional NNs

Let’s look at the **Graph Convolutional Network (GCN)** as introduced in  
[Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/pdf/1609.02907.pdf).  
Don’t worry if the details look dense — we’ll highlight the main intuition.

The layer update rule is:

$$
\mathbf{x}_u^{(\ell + 1)} =
\sum_{v \in N(u) \cup \{v\}}
\frac{1}{c_{uv}} \, \mathbf{x}_v^{(\ell)} \mathbf{W}^{(\ell + 1)}
$$

---

#### 🔍 What this means
- In the $\ell^\mathrm{th}$ GNN layer, each node $v$ updates its representation $\mathbf{x}_v^{(\ell+1)}$ by aggregating the feature vectors of its **neighbors** (and itself).  
- $\mathbf{W}^{(\ell+1)}$ a matrix of learnable weights shared across all nodes in layer $\ell+1$.  
- $c_{uv}$ is a **normalization factor** ensuring that high-degree nodes don’t dominate the aggregation.  
  In the original GCN paper:
  $$
  c_{uv} = \sqrt{(deg(u)+1)(deg(v)+1)}
  $$

---

#### 🧩 Intuition
You can think of this operation as a **weighted average of the neighborhood features**:
- Each node “looks” at its neighbors,
- Scales their features by a normalization term,
- And mixes them through a shared linear transformation.

In matrix form, this corresponds to multiplying the node feature matrix $\mathbf{X}\in\mathbb{R}^{|V|\times N}$
by a normalized adjacency matrix $\tilde{\mathbf{A}}$ — a diffusion-like smoothing that blends information locally across the graph.


## 2.2 Cora (Karate Club but bigger!)

We will now use the [Cora dataset](https://relational.fit.cvut.cz/dataset/CORA) to predict the subject of a research paper given its words and citation network. The learning task is the same as it was with Karate Club, **semi-supervised node classification**. The Cora dataset consists of

- 2,708 nodes (scientific papers)
- 7 truth classes (paper subjects)
- 5,429 edges (representing citations)
- 1,433 node features (each a binary field indicating the presence of a word in the corresponding paper)

Let's go!

In [ ]:
# import the cora citation network
from torch_geometric.datasets import Planetoid
cora_dataset = Planetoid(root='data/Planetoid', name='Cora')

In [ ]:
# there is only one graph in this dataset
cora = cora_dataset[0]
cora

**Exercise 2.2a** Print out the train, val, and test masks. How many nodes are we training with? How could this affect the prediction task?

In [ ]:
# EXERCISE



In [ ]:
# use our previous function to inspect Cora
get_graph_stats(cora)

In [ ]:
# let's visualize the graph
plt.figure(figsize=(10,10))
cora_nx = torch_geometric.utils.to_networkx(cora, to_undirected=True)
nx.draw(cora_nx, node_size=15, node_color=cora.y)

Okay, now it's time to set up a GCN using PyG! Reminder: we can construct a GNN in PyG similary to how we build a standard feed-forward neural network. We just need to swap the `torch.nn.Linear` layers with PyG's `GCNConv` implementing the graph convolution:

$$
\mathbf{x}_u^{(\ell + 1)} =
\sum_{v \in N(u) \cup \{v\}}
\frac{1}{c_{uv}} \, \mathbf{x}_v^{(\ell)} \mathbf{W}^{(\ell + 1)}
$$

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(cora_dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, cora_dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index) # apply first GCN convolution
        x = F.relu(x) # hidden graph embedding
        x = F.dropout(x, training=self.training) # randomly drop training nodes
        x = self.conv2(x, edge_index) # output graph embedding

        return x

This GCN is very similar to the [original implementation](https://github.com/tkipf/pygcn).

Let's take a look at the predictions of an untrained GCN by looking at a low dimensional representation:

In [ ]:
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

# examine the output of an untrained GCN
model = GCN()
model.eval()
out = model(cora)
visualize(out, color=cora.y)

As an aside, we've used another dimensionality reduction technique called **t-SNE (t-Distributed Stochastic Neighbor Embedding)**. It creates a low-dimensional map that preserves *local relationships* in high-dimensional data, making clusters and structure visually interpretable.

This is as good a time as any to introduce the **Adam** optimizer.  

**Adam (Adaptive Moment Estimation)** is one of the most widely used optimization algorithms in deep learning.  
It combines the strengths of **Momentum** and **RMSProp** to adapt the learning rate for each parameter individually.

Each parameter is updated as:
$$
\theta_{t+1} = \theta_t - \eta \frac{\widehat{m}_t}{\sqrt{\widehat{v}_t} + \epsilon}
$$

---

- **Momentum ($m_t$)** → smooths updates by tracking an exponential moving average of past gradients  
- **Adaptive scaling ($v_t$)** → adjusts each parameter’s step size based on its gradient variance  
- Works reliably *out of the box*, especially in settings with noisy or sparse gradients.


In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train(data):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(data):
      model.eval()
      out = model(data)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

for epoch in range(1, 101):
    loss = train(cora)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
test_acc = test(cora)
print(f'Test Accuracy: {test_acc:.4f}')

Not perfect, but it's pretty accurate at predicting paper subject area!

We can further verify that  by looking at the output embeddings of our **trained** model, which now produces a far better clustering of nodes of the same category.



In [ ]:
model.eval()

out = model(cora)
visualize(out, color=cora.y)

**Exercise 2.2b** Play around with the training loop above! Some guiding questions:

- How does GCN behave when increasing the hidden feature dimensionality or the number of layers? Does increasing the number of layers help at all?

- So far, we've just looked at GCNConv operations. You can try to use different GNN layers to see how model performance changes. What happens if you swap out all GCNConv instances with GATConv layers that make use of attention? Try to write a 2-layer GAT model that makes use of 8 attention heads in the first layer and 1 attention head in the second layer, uses a dropout ratio of 0.6 inside and outside each GATConv call, and uses a hidden_channels dimensions of 8 per head.

In [ ]:
# EXERCISE




In [ ]:
# EXERCISE




In [ ]:
# EXERCISE




## Solutions to Exercises

In [ ]:
# answers to exercises:

# Exercise 1.1a
# print(karate_club.x)
# print(karate_club.edge_index)
# print(karate_club.y)

# Exercise 1.1b
# A = torch_geometric.utils.to_dense_adj(karate_club.edge_index).squeeze()
#print('adjacency representation', A)

# Exercise 1.1c
# Batching in PyG works by concatenating nodes and creating a block diagonal adacency matrix.
# Membership is tracked through the `batch` attribute appended to each graph batch.
# See: https://pytorch-geometric.readthedocs.io/en/latest/advanced/batching.html

# Exercise 2.2a
# sum(cora.train_mask)
# There are 140 training nodes, 20 from each of the 7 truth classes.
# This means overtraining is a real possibility and regularization will be important.

# Exercise 2.2b
# Adding more layers doesn't help too much (and even hurts if the network becomes too deep)!
# This is because node features get "washed out" across the graph.

# Exercise 3a
# 21 input features, 6 output classes
# x = global_mean_pool(x, data.batch)

# Exercise 3b
# Just copy the train loop structure and the accuracy metric from the Cora train loop.
# But remember accuracy is computed per-graph now!